# Pets Sample (Multi Core TPU Sample)

<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/samples/test_fast_tpu_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq --no-cache-dir torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchtext==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 735.4MB 1.1MB/s 
     |████████████████████████████████| 12.8MB 42.3MB/s 
     |████████████████████████████████| 7.0MB 6.6MB/s 


In [5]:
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [2]:
# !pip install -Uqq git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastai==2.3.0

     |████████████████████████████████| 194kB 6.3MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 


In [3]:
!pip install -Uqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 31kB/s 
     |████████████████████████████████| 61kB 3.0MB/s 
ERROR: earthengine-api 0.1.254 has requirement google-api-python-client>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.


In [4]:
# !pip install -Uqq fastai_xla_extensions
!pip install -Uqq git+https://github.com/butchland/fastai_xla_extensions.git

In [6]:
!pip freeze | grep torch
!pip freeze | grep fast

torch==1.7.1+cu101
torch-xla==1.7
torchsummary==1.5.1
torchtext==0.8.0
torchvision==0.8.2+cu101
fastai==2.2.7
fastai-xla-extensions==0.0.8
fastcore==1.3.19
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


Start of kernel restart

In [7]:
from fastai.vision.all import *
from fastai_xla_extensions.all import *

In [8]:
# path = untar_data(URLs.MNIST_TINY)
# path = untar_data(URLs.MNIST); Path.BASE_PATH = path; path.ls()
path = untar_data(URLs.PETS)/'images'; Path.BASE_PATH = path; path.ls()

(#7393) [Path('boxer_41.jpg'),Path('american_pit_bull_terrier_72.jpg'),Path('yorkshire_terrier_185.jpg'),Path('german_shorthaired_197.jpg'),Path('Egyptian_Mau_96.jpg'),Path('shiba_inu_172.jpg'),Path('Russian_Blue_104.jpg'),Path('basset_hound_52.jpg'),Path('British_Shorthair_96.jpg'),Path('Maine_Coon_100.jpg')...]

In [9]:
bs = 64
arch = resnet34

In [10]:
pat = r'(.+)_\d+.jpg'
data = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=using_attr(RegexLabeller(pat),'name'),
    splitter=RandomSplitter(seed=42),
    item_tfms=Resize(112),
    batch_tfms=[Normalize.from_stats(*imagenet_stats)]
)

In [11]:
dls = data.dataloaders(path, bs=bs)

In [12]:
learner = cnn_learner(dls, arch, metrics=accuracy, concat_pool=False)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [13]:
%%time
learner.xla_fit(5, lr=4e-3)

start fit


epoch,train_loss,valid_loss,accuracy,time
0,0.613771,1.544855,0.662838,00:40
1,0.613933,0.755649,0.791216,00:14
2,0.631475,0.633968,0.810135,00:13
3,0.622203,0.600104,0.822297,00:12
4,0.605481,0.566854,0.825000,00:13


CPU times: user 591 ms, sys: 302 ms, total: 893 ms
Wall time: 1min 49s


In [14]:
learner.validate()

(#2) [0.5695473551750183,0.8267929553985596]

In [15]:
data2 = data.new(item_tfms=Resize(224), batch_tfms=[Normalize.from_stats(*imagenet_stats)])

In [16]:
dls2 = data2.dataloaders(path, bs=64)

In [17]:
learner.dls = dls2 # replace with size 224 sized

In [18]:
learner.xla_fit(5, lr=6e-3)

start fit


epoch,train_loss,valid_loss,accuracy,time
0,0.092213,0.433083,0.870946,01:22
1,0.119729,0.373423,0.886486,00:46
2,0.137076,0.275434,0.906757,00:42
3,0.145733,0.283003,0.914189,00:41
4,0.151304,0.306933,0.911486,00:40
